In [1]:
# scientific package imports
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from matplotlib import rcParams

import astropy.units as u
from astropy.constants import G

# import my own classes
from galaxy.galaxy import Galaxy
from galaxy.galaxies import Galaxies
from galaxy.massprofile import MassProfile
from galaxy.timecourse import TimeCourse
from galaxy.centerofmass import CenterOfMass
from galaxy.plots import Plots

In [5]:
tc = TimeCourse(usesql=True)

In [6]:
galname = 'MW'
snap = 0

In [14]:
def get_a(galname, snap, r_outer=100):
    gal = Galaxy(galname, snap, usesql=True)
    com_p, _ = tc.get_one_com(galname, snap)
    mp = MassProfile(gal, com_p)
    fitted_a, perr = mp.fit_hernquist_a(r_outer=r_outer, get_details=True)
    return fitted_a, perr

In [20]:
with open('./hqa.txt', 'w') as f:
    f.write("gal, snap, a, err\n")
for galname in ('MW','M31','M33'):
    print(galname)
    for snap in np.arange(0, 10):
        a, err = get_a(galname, snap)
        with open('./hqa.txt', 'a') as f:
            f.write(f" {galname}, {snap:03}, {a.value:6.1f}, {err.value:4.1f}\n")        

MW
M31
M33
